In [1]:
import os
import json
import gc
from unidecode import unidecode

In [2]:
wikidata_graph = {}
wikidata_name_id = {}
wikidata_en_graph = {}
wikidata_en_graph = {}
wikidata_text = {}
wikidata_en_text = {}
codex_graph = {}

In [3]:
find = ['P31', 'P279', 'P361'] # instance of, subclass of, part of, these relations are very vague so we discard them
with open('/home/vvjain3/rag-llm-verify/wikidata5m_all_triplet.txt', 'r') as file:
    for line in file:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        if line[1] in find:
            continue
        if line[0] not in wikidata_graph:
            wikidata_graph[line[0]] = {}
        if line[1] not in wikidata_graph[line[0]]:
            wikidata_graph[line[0]][line[1]] = []
        wikidata_graph[line[0]][line[1]].append(line[2])

In [4]:
wikidata_graph['Q41828']

{'P19': ['Q193722'], 'P106': ['Q39631'], 'P27': ['Q159']}

In [5]:
# #only choosing relations are unique per entity
# double_val = []
# for key, value in wikidata_graph.items():
#     new_value = {}
#     for k, v in value.items():
#         if len(v) > 1:
#             double_val.append((key, k, v))
#             continue
#         new_value[k] = v[0].strip()
#     wikidata_graph[key] = new_value
# gc.collect()

In [6]:
len(wikidata_graph)

4276852

In [7]:
with open('wikidata5m_text.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        text = line[1].strip()
        if len(line) > 2:
            for i in range(2, len(line)):
                text += ' ' + line[i].strip()
        if line[0] in wikidata_graph:
            wikidata_text[line[0]] = text
gc.collect()

0

In [8]:
name_not_in_text = []
possible_entity_names = {}
with open('wikidata5m_entity.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        if line[0] not in wikidata_text:
            continue
        possible_entity_names[line[0]] = line[1:]
        possible = line[1:min(30, len(line))] #select most common label from the first 30 aliases
        common = ''
        all_text = wikidata_text[line[0]]
        all_text = unidecode(all_text)
        all_text = all_text.lower()
        all_text = all_text.replace('–', '-') #replace en dash with hyphen
        min_ind = 1000000
        #choose common according to the order of the aliases appearing in the text
        for name in possible:
            name = unidecode(name)
            name = name.lower()
            if name in all_text:
                ind = all_text.index(name)
                if ind < min_ind:
                    min_ind = ind
                    common = name
                if min_ind <= 15:
                    break
        if common == '':
            name_not_in_text.append((line[0], all_text, line[1:], possible))
            continue
        wikidata_name_id[line[0]] = common.strip()
with open('wikidata5m_relation.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        wikidata_name_id[line[0]] = unidecode(line[1]).lower()
gc.collect()
len(name_not_in_text)

1058800

In [9]:
len(wikidata_name_id)

3129731

In [10]:
wikidata_en_graph = {}

In [11]:
wikidata_id_name = {}
for key, value in wikidata_name_id.items():
    wikidata_id_name[value] = key

In [12]:
wikidata_util = {}
for key, value in wikidata_graph.items():
    wikidata_util[key] = {}
    for rel, objs  in value.items():
        for obj in objs:
            wikidata_util[key][obj] = rel
gc.collect()
len(wikidata_util), len(wikidata_graph)

(4276852, 4276852)

In [13]:
import re

In [15]:
new_wikidata = {}
shortened = 0
total = 0
for key, value in wikidata_util.items():
    if key not in wikidata_text or key not in wikidata_name_id:
        shortened += len(list(value.keys()))
        continue
    total += len(list(value.keys()))
    all_text = wikidata_text[key]
    all_text = unidecode(all_text)
    all_text = all_text.lower()
    all_text = all_text.replace('–', '-')
    new_value = {}
    for obj, rel in value.items():
        if obj in wikidata_name_id and rel in wikidata_name_id:
            if obj in possible_entity_names:
                all_possible_vals = possible_entity_names[obj]
            else:
                all_possible_vals = [wikidata_name_id[obj]]
            for val in all_possible_vals:
                val = unidecode(val)
                val = val.lower()
                val = val.strip()
                pattern = r'\b' + re.escape(val) + r'\b'
                matches = re.search(pattern, all_text)
                if matches:
                    new_value[obj] = rel
                    break
            total += 1
    if len(new_value) > 0:
        new_wikidata[key] = new_value
gc.collect()
shortened, total

(4397801, 21949468)

In [16]:
shortened = 0
total = 0
for key, value in wikidata_util.items():
    if key not in wikidata_text or key not in wikidata_name_id:
        shortened += 1
    total += 1
shortened, total

(1147946, 4276852)

In [17]:
wikidata_graph_util = new_wikidata

In [18]:
for key, value in wikidata_graph_util.items():
    assert key in wikidata_name_id
    assert key in wikidata_text
    for k, v in value.items():
        assert k in wikidata_name_id
        assert v in wikidata_name_id
        assert k in wikidata_text

In [19]:
count = 0
total = 0
for key, value in wikidata_graph.items():
    for rel, objs in value.items():
        if len(objs) > 1:
            count += 1
        total += 1
count, total

(1380669, 13714786)

In [20]:
wiki = {}
for key, value in wikidata_graph_util.items():
    wiki[key] = {}
    for k, v in value.items():
        if v not in wiki[key]:
            wiki[key][v] = []
        wiki[key][v].append(k)
gc.collect()

0

In [21]:
count = 0
total = 0
for key, value in wiki.items():
    for rel, objs in value.items():
        if len(objs) > 1:
            count += 1
        total += 1
count, total

(206434, 4790905)

In [22]:
with open('wikidata_graphs1/wikidata_name_id.json', 'w') as f:
    json.dump(wikidata_name_id, f)
with open('wikidata_graphs1/wikidata_text.json', 'w') as f:
    json.dump(wikidata_text, f)
with open('wikidata_graphs1/wikidata_util.json', 'w') as f:
    json.dump(wikidata_graph_util, f)

In [1]:
import os
import json
from utils import *

/home/vvjain3/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-11 07:06:03.984907: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 07:06:04.034584: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 07:06:04.034616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 07:06:04.036314: E 

In [2]:
wikidata_util = json.load(open('wikidata_graphs1/wikidata_util.json'))
wikidata_name_id = json.load(open('wikidata_graphs1/wikidata_name_id.json'))
wikidata_text = json.load(open('wikidata_graphs1/wikidata_text.json'))
len(wikidata_util), len(wikidata_name_id), len(wikidata_text)

(2664259, 3129731, 4191605)

In [3]:
entity_aliases = load_aliases('wikidata5m_entity.txt')
relation_aliases = load_aliases('wikidata5m_relation.txt')

In [4]:
for key, value in wikidata_util.items():
    assert key in wikidata_name_id
    assert key in wikidata_text
    assert key in entity_aliases
    for k, v in value.items():
        assert k in wikidata_name_id
        assert v in wikidata_name_id
        assert k in wikidata_text
        assert k in entity_aliases
        assert v in relation_aliases

In [5]:
graph_algos = GraphAlgos(wikidata_util, entity_aliases, relation_aliases)

In [6]:
query_results = graph_algos.generate_random_query(4, return_path=True)
path_en = []
for i in range(len(query_results[3])):
    path_en.append(wikidata_name_id[query_results[3][i]])
print(wikidata_name_id[query_results[1]], wikidata_name_id[query_results[3][-1]], path_en, query_results[0][0])

charles lovemore mungoshi humans ['charles lovemore mungoshi', 'writer', 'ritin', 'humans'] What is the played by of the activity corresponding to this occupation of the occupation of Charles Lovemore Mungoshi?


In [7]:
some_vertex =  'Q200482' #'Q453934' #'Q36740'

subgraph = graph_algos.create_subgraph_within_radius(some_vertex, 4)
print(len(subgraph))
subgraph_algos = GraphAlgos(subgraph, entity_aliases, relation_aliases)

1438


In [8]:
all_paths = set()
all_queries = set()
distractors = {}
num_correct_answers = []
for i in range(4000):
    query_results = subgraph_algos.generate_random_query(4, return_path=True, source=some_vertex)
    distractor = subgraph_algos.get_best_distractor(query_results[1], query_results[3])
    if distractor is None:
        continue
    # print(f"Distractor: {distractor}, path: {query_results[3]}")
    num_correct_answers.append(len(query_results[2]))
    path_key = tuple(query_results[3])
    all_paths.add(path_key)
    all_queries.add(query_results[0][0])
    if path_key not in distractors:
        distractors[path_key] = set()
    distractors[path_key].add(distractor) 
    # if (i+1) % 10 == 0:
    #     print(f"Iteration: {i}, Paths: {len(all_paths)}, Queries: {len(all_queries)}, Distractors: {len(distractors)}")
all_distractors = set()
for key, value in distractors.items():
    for dis in value:  
        all_distractors.add(dis)
len(all_paths), len(all_queries), len(distractors), len(all_distractors)

(120, 1297, 120, 106)